# Generation of nonsense and frameshift mutated sequences

In [1]:
import pandas as pd
from bgvep.readers import Tabix
import re
from tqdm.notebook import trange, tqdm
import math
from Bio import SeqIO
import numpy as np

## 1. Prepare CPTAC dataset

In [19]:
####################
#Load CPTAC dataset
####################

print('Load CPTAC dataset')

CPTAC = ['BRCA', 'CCRCC', 'COAD', 'GBM', 'LUAD', 'OV', 'UCEC']
cptac_df = pd.DataFrame(columns=['gene', 'cpct_aliquot', 'protein_expression', 'aliquot', 'sample',
       'Type', 'median', 'stdev', 'norm_protein_expression', 'log2fpkm',
       'log10fpkm', '#Uploaded_variation', 'Location', 'Allele', 'Feature',
       'protein_mutation', 'Protein_position', 'Amino_acids', 'Consequence',
       'Phenotype', 'Ubiquitinases_Mutated', 'Altered_E3_Ligases',
       'Raw_Residual', 'Stability_Change', 'ABS_Stability_Change'])
for cancer in tqdm(CPTAC, total=len(CPTAC)):
    df = pd.read_csv("/workspace/projects/cptac_analysis/data/"+cancer+"/dataset_irls.gz", sep='\t')
    Dataset = [cancer] * len(df)
    df['Dataset'] = Dataset
    frames = [cptac_df, df]
    cptac_df = pd.concat(frames)
cptac_df.drop_duplicates(keep='first',inplace=True) 

cptac_df['ID'] = cptac_df['gene'].astype(str)+cptac_df['Feature'].astype(str)+cptac_df['sample'].astype(str)+cptac_df['#Uploaded_variation'].astype(str)+cptac_df['Location'].astype(str)

######################################################################################################
#Calculate the mean of rna values for the repeated mutations (log2fpkm, log10fpkm, Stability_Change)
######################################################################################################

print('Eliminate duplicated rna measures')

dupl_samples = ['C3L-00908', 'C3N-00545', 'C3N-01825']

cptac_dupl_df = cptac_df[cptac_df['sample'].isin(dupl_samples)]
df1 = cptac_dupl_df[cptac_dupl_df['Feature'].isnull()].groupby(['gene','sample'],as_index=False).mean()
df2 = cptac_dupl_df[~cptac_dupl_df['Feature'].isnull()].groupby(['gene','Feature','#Uploaded_variation','Location'],as_index=False).mean()
cptac_dupl2_df = pd.concat([df1,df2])
cptac_dupl3_df = cptac_dupl_df.drop(['log2fpkm','log10fpkm','Raw_Residual','Stability_Change','ABS_Stability_Change'],axis=1)
cptac_dupl4_df = pd.merge(cptac_dupl3_df,cptac_dupl2_df,how='left')
cptac_dupl4_df.drop_duplicates(keep='first',inplace=True)
cptac2_df = cptac_df[~(cptac_df['sample'].isin(dupl_samples))]
cptac_df = pd.concat([cptac2_df,cptac_dupl4_df],ignore_index=True)

###########################################
#Select stop_gained and frameshift_variant
###########################################

print('Select mutated sequences')

cptac_mut_df = cptac_df[['gene','Feature','sample','#Uploaded_variation','Location','Phenotype','Dataset','ID']][(cptac_df['Phenotype']=='stop_gained')|(cptac_df['Phenotype']=='frameshift_variant')]


Load CPTAC dataset



Eliminate duplicated rna measures
Select mutated sequences


## 2. Prepare CCLE dataset

In [20]:
####################
#Load CCLE dataset
####################

print('Load CCLE dataset')

ccle_df = pd.read_csv("/workspace/projects/cptac_analysis/data/CCLE/dataset_irls.gz", sep='\t')

ccle_df['ID'] = ccle_df['gene'].astype(str)+ccle_df['Feature'].astype(str)+ccle_df['sample'].astype(str)+ccle_df['#Uploaded_variation'].astype(str)+ccle_df['Location'].astype(str)

#############################################################################################################################
#Calculate the mean of protein values for the repeated mutations (protein_expression, norm_protein_expression, Raw_Residual, Stability_Change, ABS_Stability_Change)
#############################################################################################################################

print('Eliminate duplicated protein measures')

df1 = ccle_df[ccle_df['Phenotype']=='WT'].groupby(['gene','sample','Phenotype'],as_index=False).mean()
df2 = ccle_df[ccle_df['Phenotype']!='WT'].groupby(['gene','Feature','sample','#Uploaded_variation','Location','Phenotype'],as_index=False).mean()
ccle2_df = pd.concat([df1,df2])
ccle3_df = ccle_df.drop(['protein_expression', 'norm_protein_expression', 'Raw_Residual', 'Stability_Change', 'ABS_Stability_Change'],axis=1)
ccle4_df = pd.merge(ccle3_df,ccle2_df,how='left')
ccle_df = ccle4_df.drop_duplicates(keep='first')

##################################################################################
#Prepare cptac_df table for the merge(select stop_gained and frameshift_variant)
##################################################################################

print('Select mutated sequences')

ccle_mut_df = ccle_df[['gene','Feature','sample','#Uploaded_variation','Location','Phenotype','ID']][(ccle_df['Phenotype']=='stop_gained')|(ccle_df['Phenotype']=='frameshift_variant')]


Load CCLE dataset
Eliminate duplicated protein measures
Select mutated sequences


## 3. Import and prepare necessary tables

### 3.1 Import tables with wt seq (BioMart) from canonical transcripts

In [21]:
#Import wt sequences from canonical transcripts
seq_df = pd.read_csv("/workspace/users/msanchezg/notebooks/ens_canonical_sequences.tsv", sep="\t")

#Merge mutations_df with seq_df
seq_merge_df = seq_df[['ensembl_transcript_id','coding']]
seq_merge_df = seq_merge_df.rename(columns={'ensembl_transcript_id':'Feature','coding':'seq_wt'})

cptac_mut_df = pd.merge(cptac_mut_df, seq_merge_df, how='left')
ccle_mut_df = pd.merge(ccle_mut_df, seq_merge_df, how='left')

### 3.2 Import CDS positions from BG-VEP 

This step is to save the tables XXXX_cds_pos_df, because the function **cds_pos_bgvep** (inside the function **cds_pos_table**) takes a lot of time, so we run it only once and then load the saved tables for posterior analysis

In [6]:
def cds_pos_bgvep (mut_df, positions_df):
    '''Function to get cds positions for each mutation using bgvep annotation'''

    positions_list = positions_df.values.tolist()

    col_names = ['Feature','Chromosome','Position','Reference','Alternate','cDNA_position','CDS_position','Strand']

    positions_list100 = positions_list[0:100]
    cds_pos_df = pd.DataFrame(columns=col_names)
    for position in tqdm(positions_list):
        chrom = position[0]
        pos = int(position[1])
        with Tabix('hg38', '92') as reader:
            for data in reader.get(chrom, pos, pos):
                canonical = re.findall('YES', data[21])
                if canonical !=[]:
                    data2 =[]
                    data2.append(data[5])
                    data2.append(data[0])
                    data2.append(data[1])
                    data2.append(data[2])
                    data2.append(data[3])
                    data2.append(data[8])
                    data2.append(data[9])
                    data2.append(data[16])
                    newRow = pd.DataFrame([data2], columns=col_names)
                    cds_pos_df = cds_pos_df.append(newRow, ignore_index=True) 
    
    return (cds_pos_df)         

In [7]:
def cds_pos_table (mut_df):
    '''Function to create a table with cds positions of the mutations'''
    mut_df['chrom'] = mut_df['Location'].str.split(':').str[0]
    mut_df['location2'] = mut_df['Location'].str.split(':').str[1]
    mut_df['location2'] = mut_df['location2'].str.split('-').str[0]
    positions_df = mut_df[['chrom','location2']]
    cds_pos_df = cds_pos_bgvep (mut_df, positions_df)
    cds_pos_df = cds_pos_df.rename(columns={'Position':'location2'})
    cds_pos_df['location2'] = cds_pos_df['location2'].astype(str)
    cds_pos_df['chrom'] = 'chr' + cds_pos_df['Chromosome'].astype(str)
    cds_pos_df = cds_pos_df.drop(columns=['Chromosome','Alternate'])
    cds_pos_df.drop_duplicates(keep='first',inplace=True)
    return (cds_pos_df)

In [9]:
#Add cds positions to datasets (takes a lot of time, 1-5h)
cptac_cds_pos_df = cds_pos_table (cptac_mut_df)
ccle_cds_pos_df = cds_pos_table (ccle_mut_df)

In [19]:
#Save tables
cptac_cds_pos_df.to_csv(r'cds_pos_nsfs_cptac.tsv', header = True, index = None, sep = '\t')
ccle_cds_pos_df.to_csv(r'cds_pos_nsfs_ccle.tsv', header = True, index = None, sep = '\t')

From here we can continue after uploading the saved XXXX_cds_pos_df tables

In [23]:
#Load tables
cptac_cds_pos_df = pd.read_csv('cds_pos_nsfs_cptac.tsv', sep='\t')
ccle_cds_pos_df = pd.read_csv('cds_pos_nsfs_ccle.tsv', sep='\t')

In [29]:
#Merge mutations_df with cds_pos_df

cptac_cds_pos_df['location2'] = cptac_cds_pos_df['location2'].astype(str)
cptac_mut_df['chrom'] = cptac_mut_df['Location'].str.split(':').str[0]
cptac_mut_df['location2'] = cptac_mut_df['Location'].str.split(':').str[1]
cptac_mut_df['location2'] = cptac_mut_df['location2'].str.split('-').str[0]
cptac_mut_df = pd.merge(cptac_mut_df,cptac_cds_pos_df, how='left')

ccle_cds_pos_df['location2'] = ccle_cds_pos_df['location2'].astype(str)
ccle_mut_df['chrom'] = ccle_mut_df['Location'].str.split(':').str[0]
ccle_mut_df['location2'] = ccle_mut_df['Location'].str.split(':').str[1]
ccle_mut_df['location2'] = ccle_mut_df['location2'].str.split('-').str[0]
ccle_mut_df = pd.merge(ccle_mut_df,ccle_cds_pos_df, how='left')


In [32]:
#Select mutations with CDS positions for in silico mutational process
cptac_mut_df = cptac_mut_df[cptac_mut_df['CDS_position']!='-']
cptac_mut_df = cptac_mut_df[~cptac_mut_df['CDS_position'].isnull()]
ccle_mut_df = ccle_mut_df[ccle_mut_df['CDS_position']!='-']
ccle_mut_df = ccle_mut_df[~ccle_mut_df['CDS_position'].isnull()]

## 4. Create mutated sequences

In [34]:
def mutate_seq (df):
    '''Function to generate the mutated sequences with nonsense and frameshift alterations'''
    
    df['REF'] = df['#Uploaded_variation'].str.split('__').str[1]
    df['ALT'] = df['#Uploaded_variation'].str.split('__').str[2]
    
    mutations_list = df[['seq_wt','CDS_position','REF','ALT','Reference','Strand','Feature','sample','Location','#Uploaded_variation','Phenotype','ID']].values.tolist()

    def fix_nt (nt, strand):
        compl_dict = {'A':'T','T':'A','C':'G','G':'C'}
        if strand == -1: 
            if len(nt)>1:
                nt_list = list(nt)
                nt2_list = []
                for base in nt_list:
                    nt2 = compl_dict[base]
                    nt2_list.append(nt2)
                nt2 = ''.join(nt2_list)
            else:
                nt2 = compl_dict[nt]
        else:
            nt2 = nt
        return(nt2)

    mut_check_list =[]
    seq_mut_list =[]
    for seq in mutations_list:
        ref = seq[2]
        alt = seq[3]
        ref_b = seq[4]
        strand = seq[5]
        mutation = seq[10]
        if seq[0] is np.nan:
            mut_check = [seq[6],seq[7],seq[8],seq[9],'nan','nan','nan','nan','nan','nan','nan','nan','nan','nan','nan','nan','nan',seq[11]]
            seq_mut2 = [seq[6],seq[7],seq[8],seq[9],seq[10],ref,alt,seq[0],seq[11]]
        else:
            if (ref!='-')&(alt!='-'):
                ref2 = fix_nt(ref, strand)
                alt2 = fix_nt(alt, strand)
                ref2_list = list(ref2)
                cds_pos = int(seq[1])
                seq_wt = seq[0]
                if len(seq_wt)<cds_pos:
                    mut_check = [seq[6],seq[7],seq[8],seq[9],'nan','nan',ref,ref_b,ref2,'nan',alt,alt2, len(seq_wt),'nan',len(ref2),len(alt2),'nan',seq[11]]
                    seq_mut2 = [seq[6],seq[7],seq[8],seq[9],seq[10],ref,alt,'Short_seq_wt',seq[11]]
                else:
                    ref3 = seq_wt[cds_pos-1]
                    if ref2_list[0] != ref3:
                        mut_check = [seq[6],seq[7],seq[8],seq[9],'nan','nan',ref,ref_b,ref2,ref3,alt,alt2,len(seq_wt),'nan',len(ref2), len(alt2),'nan',seq[11]]
                        seq_mut2 = [seq[6],seq[7],seq[8],seq[9],seq[10],ref,alt,'No_ref_match',seq[11]]
                    else:
                        seq_mut = seq_wt[0:(cds_pos-1)]+alt2+seq_wt[cds_pos:len(seq_wt)]
                        seq_mut2 = [seq[6],seq[7],seq[8],seq[9],seq[10],ref,alt,seq_mut,seq[11]]
                    mut_check = [seq[6],seq[7],seq[8],seq[9],seq_wt[(cds_pos-2):(cds_pos+3)],seq_mut[(cds_pos-2):(cds_pos+3)],ref,ref_b,ref2,ref3,alt,alt2,len(seq_wt), len(seq_mut), len(ref2), len(alt2), (len(seq_mut)-len(seq_wt)), seq[11]]
            elif alt!='-':
                ref2 = fix_nt(ref_b, strand)
                alt2 = fix_nt(alt, strand)
                cds_pos = int(seq[1])
                seq_wt = seq[0]
                if len(seq_wt)<cds_pos:
                    mut_check = [seq[6],seq[7],seq[8],seq[9],'nan','nan',ref,ref_b,ref2,'nan',alt,alt2, len(seq_wt),'nan',len(ref2),len(alt2),'nan',seq[11]]
                    seq_mut2 = [seq[6],seq[7],seq[8],seq[9],seq[10],ref,alt,'Short_seq_wt',seq[11]]
                else:
                    ref3 = seq_wt[cds_pos-1]
                    if ref2 != ref3:
                        mut_check = [seq[6],seq[7],seq[8],seq[9],'nan','nan',ref,ref_b,ref2,ref3,alt,alt2,len(seq_wt),'nan',len(ref2), len(alt2),'nan',seq[11]]
                        seq_mut2 = [seq[6],seq[7],seq[8],seq[9],seq[10],ref,alt,'No_ref_match',seq[11]]
                    else:
                        seq_mut = seq_wt[0:(cds_pos-1)]+alt2+seq_wt[(cds_pos-1):len(seq_wt)]
                        seq_mut2 = [seq[6],seq[7],seq[8],seq[9],seq[10],ref,alt,seq_mut,seq[11]]
                        mut_check = [seq[6],seq[7],seq[8],seq[9],seq_wt[(cds_pos-2):(cds_pos+3)],seq_mut[(cds_pos-2):(cds_pos+len(alt2)+3)],ref,ref_b,ref2,ref3,alt,alt2,len(seq_wt), len(seq_mut), len(ref2), len(alt2), (len(seq_mut)-len(seq_wt)),seq[11]]
            else:
                ref2 = fix_nt(ref, strand)
                alt2 = '-'
                ref2_list = list(ref2)
                cds_pos = int(seq[1])
                seq_wt = seq[0]
                if len(seq_wt)<cds_pos:
                    mut_check = [seq[6],seq[7],seq[8],seq[9],'nan','nan',ref,ref2,'nan',alt,alt2, len(seq_wt),'nan',len(ref2),len(alt2),'nan',seq[11]]
                    seq_mut2 = [seq[6],seq[7],seq[8],seq[9],seq[10],ref,alt,'Short_seq_wt',seq[11]]
                else:
                    ref3 = seq_wt[cds_pos-1]
                    if ref2_list[0] != ref3:
                        mut_check = [seq[6],seq[7],seq[8],seq[9],'nan','nan',ref,ref_b,ref2,ref3,alt,alt2,len(seq_wt),'nan',len(ref2), len(alt2),'nan',seq[11]]
                        seq_mut2 = [seq[6],seq[7],seq[8],seq[9],seq[10],ref,alt,'No_ref_match',seq[11]]
                    else:
                        seq_mut = seq_wt[0:(cds_pos-1)]+seq_wt[(cds_pos+len(ref2)-1):len(seq_wt)]
                        seq_mut2 = [seq[6],seq[7],seq[8],seq[9],seq[10],ref,alt,seq_mut,seq[11]]
                        mut_check = [seq[6],seq[7],seq[8],seq[9],seq_wt[(cds_pos-2):(cds_pos+3)],seq_mut[(cds_pos-2):(cds_pos+3)],ref,ref_b,ref2,ref3,alt,alt2,len(seq_wt), len(seq_mut), len(ref2), len(alt2), (len(seq_mut)-len(seq_wt)),seq[11]]

        mut_check_list.append(mut_check)
        seq_mut_list.append(seq_mut2)

    mut_check_df = pd.DataFrame(mut_check_list, columns=['Feature','sample','Location','#Uploaded_cariation','seq_wt','seq_mut','ref','ref_bgveps','ref_at_strand','ref_at_seq','alt','alt_at_strand','length_seq_wt','length_seq_mut','length_ref','length_alt','length_seq_mut_minus_length_seq_wt','ID'])
    seq_mut_df = pd.DataFrame(seq_mut_list, columns=['Feature','sample','Location','#Uploaded_variation','Phenotype','REF','ALT','seq_mut','ID'])
    return (mut_check_df, seq_mut_df)

In [35]:
#Generate mutated sequences
cptac_mutcheck_df, cptac_seq_mut_df = mutate_seq(cptac_mut_df)
ccle_mutcheck_df, ccle_seq_mut_df = mutate_seq(ccle_mut_df)

In [38]:
#Table to check if in silico mutations have been done correctly
cptac_mutcheck_df

,Feature,sample,Location,#Uploaded_cariation,seq_wt,seq_mut,ref,ref_bgveps,ref_at_strand,ref_at_seq,alt,alt_at_strand,length_seq_wt,length_seq_mut,length_ref,length_alt,length_seq_mut_minus_length_seq_wt,ID
0,ENST00000645829,11BR016,chrX:75060263,11BR016__G__C,GCACA,GGACA,G,G,C,C,C,G,2274,2274,1,1,0,ABCB7ENST0000064582911BR01611BR016__G__CchrX:7...
1,ENST00000645237,20BR006,chr13:95166186,20BR006__A__T,TTGAA,TAGAA,A,A,T,T,T,A,3978,3978,1,1,0,ABCC4ENST0000064523720BR00620BR006__A__Tchr13:...
2,ENST00000376142,18BR003,chr10:26823179,18BR003__G__A,TCAGC,TTAGC,G,G,C,C,A,T,1527,1527,1,1,0,ABI1ENST0000037614218BR00318BR003__G__Achr10:2...
3,ENST00000616317,14BR008,chr17:37274257-37274269,14BR008__ATACGTTTTGAAA__-,ATCTT,ACCAG,ATACGTTTTGAAA,A,TATGCAAAACTTT,T,-,-,7152,7139,13,1,-13,ACACAENST0000061631714BR00814BR008__ATACGTTTTG...
4,ENST00000642319,11BR028,chr1:180397563,11BR028__C__A,GGAAC,GTAAC,C,C,G,G,A,T,849,849,1,1,0,ACBD6ENST0000064231911BR02811BR028__C__Achr1:1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16396,ENST00000442510,C3N-01825,chr1:198748182-198748183,C3N-01825__-__AACTCCT,AATTC,AAACTCCTATTC,-,A,A,A,AACTCCT,AACTCCT,3921,3928,1,7,7,PTPRCENST00000442510C3N-01825C3N-01825__-__AAC...
16397,ENST00000244040,C3N-01825,chr20:58354224-58354225,C3N-01825__-__AAAAAAA,GCAAA,GAAAAAAACAAA,-,C,C,C,AAAAAAA,AAAAAAA,585,592,1,7,7,RAB22AENST00000244040C3N-01825C3N-01825__-__AA...
16398,ENST00000004531,C3N-01825,chr8:17548817-17548818,C3N-01825__-__ATTCA,TTCTC,TATTCATCTC,-,T,T,T,ATTCA,ATTCA,2097,2102,1,5,5,SLC7A2ENST00000004531C3N-01825C3N-01825__-__AT...
16399,ENST00000261716,C3N-01825,chr17:29510944-29510945,C3N-01825__-__TTCATTG,TCGAG,TTTCATTGCGAG,-,C,C,C,TTCATTG,TTCATTG,3006,3013,1,7,7,TAOK1ENST00000261716C3N-01825C3N-01825__-__TTC...


In [39]:
#Table to check if in silico mutations have been done correctly
ccle_mutcheck_df

,Feature,sample,Location,#Uploaded_cariation,seq_wt,seq_mut,ref,ref_bgveps,ref_at_strand,ref_at_seq,alt,alt_at_strand,length_seq_wt,length_seq_mut,length_ref,length_alt,length_seq_mut_minus_length_seq_wt,ID
0,ENST00000262461,MCF7_BREAST,chr5:128180952-128180953,ACH-000019__-__A,TTCAT,TATCAT,-,T,T,T,A,A,3639,3640,1,1,1,SLC12A2ENST00000262461MCF7_BREASTACH-000019__-...
1,ENST00000262461,GB1_CENTRAL_NERVOUS_SYSTEM,chr5:128184383-128184384,ACH-000738__AA__-,GAAAT,GATCA,AA,A,AA,A,-,-,3639,3637,2,1,-2,SLC12A2ENST00000262461GB1_CENTRAL_NERVOUS_SYST...
2,ENST00000262461,SNGM_ENDOMETRIUM,chr5:128178638,ACH-000974__A__-,GAAAA,GAAAA,A,A,A,A,-,-,3639,3638,1,1,-1,SLC12A2ENST00000262461SNGM_ENDOMETRIUMACH-0009...
3,ENST00000262461,SKES1_BONE,chr5:128131080,ACH-000087__T__-,ATTTA,ATTAA,T,T,T,T,-,-,3639,3638,1,1,-1,SLC12A2ENST00000262461SKES1_BONEACH-000087__T_...
4,ENST00000262461,MFE319_ENDOMETRIUM,chr5:128135792-128135793,ACH-000988__-__T,GGTTT,GTGTTT,-,G,G,G,T,T,3639,3640,1,1,1,SLC12A2ENST00000262461MFE319_ENDOMETRIUMACH-00...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14911,ENST00000273371,NCIH446_LUNG,chr3:119608918,ACH-000800__G__T,CGAGA,CTAGA,G,G,G,G,T,T,1371,1371,1,1,0,PLA1AENST00000273371NCIH446_LUNGACH-000800__G_...
14912,ENST00000378076,NCIH650_LUNG,chr10:15688006,ACH-000945__C__A,AGAAC,ATAAC,C,C,G,G,A,T,3192,3192,1,1,0,ITGA8ENST00000378076NCIH650_LUNGACH-000945__C_...
14913,ENST00000222598,HEC59_ENDOMETRIUM,chr7:97022295-97022296,ACH-000994__AA__-,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,DLX5ENST00000222598HEC59_ENDOMETRIUMACH-000994...
14914,ENST00000222598,HEC59_ENDOMETRIUM,chr7:97024466,ACH-000994__C__-,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,DLX5ENST00000222598HEC59_ENDOMETRIUMACH-000994...


In [40]:
#Add mutated sequence to cptac_mut_df
cptac_mut_df = pd.merge(cptac_mut_df,cptac_seq_mut_df,how='left')
ccle_mut_df = pd.merge(ccle_mut_df,ccle_seq_mut_df,how='left')

## 5. Translation

In [42]:
genetic_code = {'GAA': 'E', 'CGA': 'R', 'GTG': 'V', 'TAA': '*', 'CGT': 'R', 'ATA': 'I', 'GAC': 'D', 'TCG': 'S', 
                  'GAT': 'D', 'ATG': 'M', 'CTG': 'L', 'CTA': 'L', 'TAC': 'Y', 'GGA': 'G', 'CGG': 'R', 'AGC': 'S', 
                  'TCT': 'S', 'TGA': '*', 'AAA': 'K', 'ACC': 'T', 'ACA': 'T', 'TGC': 'C', 'AAG': 'K', 'GTC': 'V', 
                  'TCC': 'S', 'ACT': 'T', 'AGA': 'R', 'CTT': 'L', 'GCC': 'A', 'GTA': 'V', 'TAG': '*', 'CAA': 'Q', 
                  'CAC': 'H', 'GCT': 'A', 'TTA': 'L', 'CAT': 'H', 'CGC': 'R', 'TTC': 'F', 'ATT': 'I', 'GGC': 'G', 
                  'CAG': 'Q', 'AAC': 'N', 'CCC': 'P', 'GTT': 'V', 'AGG': 'R', 'TGT': 'C', 'CCG': 'P', 'GGG': 'G', 
                  'ATC': 'I', 'TTT': 'F', 'AAT': 'N', 'TCA': 'S', 'GAG': 'E', 'CCA': 'P', 'GCA': 'A', 'TAT': 'Y', 
                  'GGT': 'G', 'TGG': 'W', 'GCG': 'A', 'CTC': 'L', 'TTG': 'L', 'CCT': 'P', 'ACG': 'T', 'AGT': 'S'}

def translation(nt_seq_list):
    '''Function to translate in silico a list of cdna sequences'''
    prot_seq_list = []
    for nt_seq in tqdm(nt_seq_list):
        if (nt_seq[0] is np.nan)|(nt_seq[0] == 'Sequence unavailable')|(nt_seq[0] == 'No_ref_match')|(nt_seq[0] == 'Short_seq_wt'):
            prot_seq_ids = [nt_seq[1],nt_seq[2],nt_seq[3],nt_seq[0]]
            prot_seq_list.append(prot_seq_ids)
        else:
            nt_seq_triplets = [nt_seq[0][i:i+3] for i in range(0, len(nt_seq[0]), 3)]
            prot_seq = ''

            for triplet in nt_seq_triplets:
                if len(triplet) == 3:
                    if 'N' in triplet:
                        prot_seq += 'X' 
                    else:
                        prot_seq += genetic_code[triplet]
                        if '*' in prot_seq:
                            break
                else:
                    break
            prot_seq_ids = [nt_seq[1],nt_seq[2],nt_seq[3],prot_seq]
            prot_seq_list.append(prot_seq_ids)
    return(prot_seq_list)

In [43]:
def translation_mut_table (df):
    '''Function to prepare a table with translated mutated sequences'''
    seq_mut_list = df[['seq_mut','Feature','#Uploaded_variation','Location']].values.tolist()
    prot_seq_mut_list = translation(seq_mut_list)
    prot_seq_mut_df = pd.DataFrame(prot_seq_mut_list, columns=['Feature','#Uploaded_variation','Location','prot_seq'])
    prot_mut_df = pd.merge(df,prot_seq_mut_df,how='left')
    return (prot_mut_df)

In [44]:
#Translate sequences
cptac_prot_mut_df = translation_mut_table(cptac_mut_df)
ccle_prot_mut_df = translation_mut_table(ccle_mut_df)

In [45]:
def translation_wt_table (df,seq_df):
    '''Function to prepare a table with translated wt sequences'''
    #Prepare cptac_df table for the merge
    wt_df = df[['gene','Feature','#Uploaded_variation','Location','protein_mutation','Protein_position','Amino_acids','sample']][df['Phenotype']!='WT']
    wt_df.drop_duplicates(keep='first',inplace=True)

    #Merge seq with cptac table for all proteins
    seq_wt_df = seq_df[['ensembl_transcript_id','coding']]
    seq_wt_df.drop_duplicates(keep='first',inplace=True)
    seq_wt_df = seq_wt_df.rename(columns={'ensembl_transcript_id':'Feature','coding':'seq_wt'})

    wt_df = pd.merge(wt_df, seq_wt_df, how='left')

    #Translate wt sequences
    seq_wt_list = wt_df[['seq_wt','Feature','#Uploaded_variation','Location']].values.tolist()
    prot_seq_wt_list = translation(seq_wt_list)
    prot_wt_df = pd.DataFrame(prot_seq_wt_list, columns=['Feature','#Uploaded_variation','Location','prot_seq'])
    return (prot_wt_df)

In [46]:
#Create a table with translated sequences
cptac_prot_wt_df = translation_wt_table (cptac_df,seq_df)
ccle_prot_wt_df = translation_wt_table (ccle_df,seq_df)

/home/msanchezg/miniconda3/envs/cptac/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


## 6. Search for c-term degrons in the generated sequences

In [53]:
def find_cdegron(prot_list, cdegron_motif):
    """This function finds all proteins containing c-terminal degrons (cdegrons)"""
    cdegron_seq_list = [] 
    for mutation in prot_list:
        seq = mutation[0]
        splitted = re.split('\*', str(seq))
        seq2 = splitted[0]
        motif_re = cdegron_motif +'$'
        motif_re = motif_re.replace('X', '.')
        find_cdegron_motif = re.findall(motif_re, seq2)
        if find_cdegron_motif != []:
            mutation_list = []
            mutation_list.append(mutation[1])
            mutation_list.append(mutation[2])
            mutation_list.append(mutation[3])
            mutation_list.append(seq)
            mutation_list.append(True)
            cdegron_seq_list.append(mutation_list)
    cdegron_df = pd.DataFrame(cdegron_seq_list,columns=['Feature','#Uploaded_variation','Location','prot_seq',cdegron_motif])
    return cdegron_df

In [54]:
cdegron_motifs = ['GG', 'RG', 'PG', 'XR', 'RXXG', 'EE', 'RXX', 'VX', 'AX', 'A']

def find_cdegron_list(prot_list, cdegron_motifs):
    """This function finds all cdegron motifs provided in a list"""
    col_names=['Feature','#Uploaded_variation','Location','prot_seq']
    all_cdegron_df = pd.DataFrame(columns=col_names)
    for motif in cdegron_motifs:
        cdegron_df = find_cdegron(prot_list, motif)
        all_cdegron_df = pd.concat([all_cdegron_df,cdegron_df])
    all_cdegron_df = all_cdegron_df.fillna(False)
    dupl = all_cdegron_df[all_cdegron_df[['Feature','#Uploaded_variation','Location']].duplicated()].values.tolist()
    df2 = pd.DataFrame()
    for seq in dupl:
        df1 = all_cdegron_df[(all_cdegron_df['Feature']==seq[0])&(all_cdegron_df['#Uploaded_variation']==seq[1])&(all_cdegron_df['Location']==seq[2])].groupby(['Feature','#Uploaded_variation','Location','prot_seq'], as_index=False).sum()
        df2 = pd.concat([df2,df1])
    all_cdegron_df.drop_duplicates(subset=['Feature','#Uploaded_variation','Location','prot_seq'], keep=False, inplace=True)
    df2.drop_duplicates(keep='first', inplace=True)
    all_cdegron_df = pd.concat([all_cdegron_df,df2],ignore_index=True)
    return all_cdegron_df

In [55]:
def cdegron_table (prot_mut_df, prot_wt_df, cdegron_motifs, df,seq_df):
    '''This function creates a table with c-degrons annotated per sequence'''
    
    #Search cdegron motifs in cptac_mut_df
    print('Search cdegron motifs in cptac_mut_df')
    
    prot_seq_mut_list = prot_mut_df[['prot_seq','Feature','#Uploaded_variation','Location']].values.tolist()
    cdegron_mut_df = find_cdegron_list(prot_seq_mut_list,cdegron_motifs)

    #Add proteins without cterm degrons to the table
    print('Add proteins without cterm degrons to the table')
    
    prot_seq_mut_df = prot_mut_df[['gene','Feature','#Uploaded_variation','Location','prot_seq']]
    cdegron_mut_df = pd.merge(prot_seq_mut_df,cdegron_mut_df,how='left')
    cdegron_mut_df = cdegron_mut_df.fillna(False)
    
    #Find wt proteins with cterm degrons
    print('Find wt proteins with cterm degrons')
    
    prot_wt_list = prot_wt_df[['prot_seq','Feature','#Uploaded_variation','Location']].values.tolist()
    cdegron_wt_df = find_cdegron_list(prot_wt_list,cdegron_motifs)

    #Add proteins without cterm degrons to the table
    print('Add proteins without cterm degrons to the table')
    
    cdegron_wt_df = pd.merge(prot_wt_df,cdegron_wt_df,how='left')
    cdegron_wt_df = cdegron_wt_df.fillna(False)

    #Add 'gene' and 'Phenotype' to cdegron_wt_df
    print('Add gene and Phenotype to cdegron_wt_df')
    
    id_df = df[['gene','Feature']][df['Phenotype']!='WT']
    id_df.drop_duplicates(keep='first',inplace=True)
    cdegron_wt_df = pd.merge(cdegron_wt_df,id_df,how='left')

    cdegron_wt2_df = cdegron_wt_df.drop(columns=['Feature','#Uploaded_variation','Location'])
    phenotype_wt = 'WT'
    cdegron_wt2_df['Phenotype'] = phenotype_wt

    #Add phenotype to cdegron_mut_df
    print('Add phenotype to cdegron_mut_df')
    
    phenotype_mut = df[['Feature','#Uploaded_variation','Location','Phenotype']][(df['Phenotype']=='stop_gained')|(df['Phenotype']=='frameshift_variant')]
    cdegron_mut_df = pd.merge(cdegron_mut_df,phenotype_mut,how='left')
    cdegron_mut_df = pd.merge(cdegron_mut_df,id_df,how='left')

    #Concat wt and mut
    print('Concat wt and mut')
    
    cdegron_df = pd.concat([cdegron_wt2_df,cdegron_mut_df])
    cdegron_df.drop_duplicates(keep='first',inplace=True) 
    return (cdegron_df)

In [56]:
#Create tables with c-degron annotations
print('CPTAC dataset')
cptac_cdegron_df = cdegron_table (cptac_prot_mut_df, cptac_prot_wt_df, cdegron_motifs, cptac_df, seq_df)
print('\nCCLE dataset')
ccle_cdegron_df = cdegron_table (ccle_prot_mut_df, ccle_prot_wt_df, cdegron_motifs, ccle_df, seq_df)

CPTAC dataset
Search cdegron motifs in cptac_mut_df
Add proteins without cterm degrons to the table
Find wt proteins with cterm degrons
Add proteins without cterm degrons to the table
Add gene and Phenotype to cdegron_wt_df
Add phenotype to cdegron_mut_df
Concat wt and mut

CCLE dataset
Search cdegron motifs in cptac_mut_df
Add proteins without cterm degrons to the table
Find wt proteins with cterm degrons
Add proteins without cterm degrons to the table
Add gene and Phenotype to cdegron_wt_df
Add phenotype to cdegron_mut_df
Concat wt and mut


In [90]:
cptac_cdegron_df

,prot_seq,GG,RG,PG,XR,RXXG,EE,RXX,VX,AX,A,gene,Phenotype,Feature,#Uploaded_variation,Location
0,MGKNKLLHPSLVLLLLVLLPTDASVSGKPQYMVLVPSLLHTETTEK...,False,False,False,False,False,False,False,False,False,True,A2M,WT,NaN,NaN,NaN
5,MWAQLLLGMLALSPAIAEELPNYLVTLPARLNFPSVQKVCLDLSPG...,False,False,False,False,False,False,False,False,False,False,A2ML1,WT,NaN,NaN,NaN
10,MCSLGLFPPPPPRGQVTLYEHNNELVTGSSYESPPPDFRGQWINLP...,False,False,False,False,False,False,False,False,False,False,AAAS,WT,NaN,NaN,NaN
12,MSKEERPGREEILECQVMWEPDSKKNTQMDRFRAAVGAACGLALES...,False,False,False,False,False,False,False,False,False,False,AACS,WT,NaN,NaN,NaN
15,MAAPAPVTRQVSGAAALVPAPSGPDSGQPLAAAVAELPVLDARGQR...,False,False,False,False,False,False,False,False,False,False,AAED1,WT,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16396,MTMYLWLKLLAFGFAFLDTEVFVTGQSPTPSPTGLTTAKMPSVPLS...,False,False,False,False,False,False,False,False,False,False,PTPRC,frameshift_variant,ENST00000442510,C3N-01825__-__AACTCCT,chr1:198748182-198748183
16397,MALRELKVCLLGDTGVGKSSIVWRFVEDSFDPNINPTIGASFMTKT...,False,False,False,False,False,False,False,False,False,False,RAB22A,frameshift_variant,ENST00000244040,C3N-01825__-__AAAAAAA,chr20:58354224-58354225
16398,MKIETSGYNSDKLICRGFIGTPAPPVCDSKFLLSPSSDVRMIPCRA...,False,False,False,False,False,False,False,False,False,False,SLC7A2,frameshift_variant,ENST00000004531,C3N-01825__-__ATTCA,chr8:17548817-17548818
16399,MPSTNRAGSLKDPEIAELFFKEDPEKLFTDLREIGHGSFGAVYFAR...,False,False,False,False,False,False,False,False,False,False,TAOK1,frameshift_variant,ENST00000261716,C3N-01825__-__TTCATTG,chr17:29510944-29510945


In [57]:
ccle_cdegron_df

,prot_seq,GG,RG,PG,XR,RXXG,EE,RXX,VX,AX,A,gene,Phenotype,Feature,#Uploaded_variation,Location
0,MEPRPTAPSSGAPGLAGVGETPSAAALAAARVELPGTAVPSVPEDA...,False,False,False,False,False,False,False,False,False,False,SLC12A2,WT,NaN,NaN,NaN
32,MLSGKKAAAAAAAAAAAATGTEAGPGTAGGSENGSEVAAQPAGLSG...,False,False,False,False,False,False,False,False,False,False,KDM1A,WT,NaN,NaN,NaN
46,MYNMMETELKPPGPQQTSGGGGGNSTAAAAGGNQKNSPDRVKRPMN...,False,False,False,False,False,False,False,False,False,False,SOX2,WT,NaN,NaN,NaN
58,MDSAAAAFALDKPALGPGPPPPPPALGPGDCAQARKNFSVSHLLDL...,False,False,False,False,False,False,False,True,False,False,PRRX2,WT,NaN,NaN,NaN
59,MVLLESEQFLTELTRLFQKCRTSGSVYITLKKYDGRTKPIPKKGTV...,False,False,False,False,False,False,False,False,True,False,SRP14,WT,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14911,MPPGPWESCFWVGGLILWLSVGSSGDAPPTPQPKCADFQSANLFEG...,False,False,False,False,False,False,False,False,False,False,PLA1A,stop_gained,ENST00000273371,ACH-000800__G__T,chr3:119608918
14912,MSPGASRGPRGSQAPLIAPLCCAAAALGMLLWSPACQAFNLDVEKL...,False,False,False,False,False,False,False,False,False,False,ITGA8,stop_gained,ENST00000378076,ACH-000945__C__A,chr10:15688006
14913,False,False,False,False,False,False,False,False,False,False,False,DLX5,frameshift_variant,ENST00000222598,ACH-000994__AA__-,chr7:97022295-97022296
14914,False,False,False,False,False,False,False,False,False,False,False,DLX5,frameshift_variant,ENST00000222598,ACH-000994__C__-,chr7:97024466


In [74]:
#Save table with cdegron annotation
cptac_cdegron_df.to_csv(r'cdegron_wtnsfs_annot_cptac.tsv', header = True, index = None, sep = '\t')
ccle_cdegron_df.to_csv(r'cdegron_wtnsfs_annot_ccle.tsv', header = True, index = None, sep = '\t')

## 7. Get c-terminal amino acid (last aa)

In [75]:
def get_last_aa(prot_list, aa):
    """This function finds all proteins containing c-terminal degrons (cdegrons)"""
    aa_seq_list = [] 
    for mutation in prot_list:
        seq = mutation[0]
        splitted = re.split('\*', str(seq))
        seq2 = splitted[0]
        aa_re = aa +'$'
        find_aa = re.findall(aa_re, seq2)
        if find_aa != []:
            mutation_list = []
            mutation_list.append(mutation[1])
            mutation_list.append(mutation[2])
            mutation_list.append(mutation[3])
            mutation_list.append(seq)
            mutation_list.append(aa)
            aa_seq_list.append(mutation_list)
    aa_df = pd.DataFrame(aa_seq_list,columns=['Feature','#Uploaded_variation','Location','prot_seq','last_aa'])
    return aa_df


In [76]:
aa_list = ['A','C','D','E','F','G','H','I','K','Q','L','M','N','P','R','S','T','V','W','Y']

def get_last_aa_list(prot_list, aa_list):
    """This function finds all cdegron motifs provided in a list"""
    col_names=['Feature','#Uploaded_variation','Location','prot_seq']
    all_aa_df = pd.DataFrame()
    for aa in aa_list:
        aa_df = get_last_aa(prot_list, aa)
        all_aa_df = pd.concat([all_aa_df,aa_df],ignore_index=True)
    return all_aa_df

In [80]:
def last_aa_table (prot_mut_df,aa_list, prot_wt_df, df):
    '''This function annotates all c-terminal amino acids (last aa) from mutated and wt sequences'''
    
    prot_seq_mut_list = prot_mut_df[['prot_seq','Feature','#Uploaded_variation','Location']].values.tolist()
    aa_mut_df = get_last_aa_list(prot_seq_mut_list,aa_list)

    #Get last aa from wt sequences
    prot_wt_list = prot_wt_df[['prot_seq','Feature','#Uploaded_variation','Location']].values.tolist()
    aa_wt_df = get_last_aa_list(prot_wt_list,aa_list)

    #Add 'gene' and 'Phenotype'
    id_df = cptac_df[['gene','Feature']][cptac_df['Phenotype']!='WT']
    id_df.drop_duplicates(keep='first',inplace=True)
    aa_wt_df = pd.merge(aa_wt_df,id_df,how='left')

    aa_wt2_df = aa_wt_df.drop(columns=['Feature','#Uploaded_variation','Location'])
    phenotype_wt = 'WT'
    aa_wt2_df['Phenotype'] = phenotype_wt

    phenotype_mut = df[['Feature','#Uploaded_variation','Location','Phenotype']][(df['Phenotype']=='stop_gained')|(df['Phenotype']=='frameshift_variant')]
    aa_mut_df = pd.merge(aa_mut_df,phenotype_mut,how='left')
    aa_mut_df = pd.merge(aa_mut_df,id_df,how='left')

    #concat wt and mut
    aa_df = pd.concat([aa_wt2_df,aa_mut_df])
    aa_df.drop_duplicates(keep='first',inplace=True) 
    return (aa_df)

In [81]:
#Create tables with last aa annotation
cptac_aa_df = last_aa_table (cptac_prot_mut_df, aa_list, cptac_prot_wt_df, cptac_df)
ccle_aa_df = last_aa_table (ccle_prot_mut_df, aa_list, ccle_prot_wt_df, ccle_df)

In [88]:
cptac_aa_df

,prot_seq,last_aa,gene,Phenotype,Feature,#Uploaded_variation,Location
0,MGKNKLLHPSLVLLLLVLLPTDASVSGKPQYMVLVPSLLHTETTEK...,A,A2M,WT,NaN,NaN,NaN
5,MRGPPAWPLRLLEPPSPAEPGRLLPVACVWAAASRVPGSLSPFTGL...,A,ABCB10,WT,NaN,NaN,NaN
8,MDALCGSGELGSKFWDSNLSVHTENPDLTPCFQNSLLAWVPCIYLW...,A,ABCC3,WT,NaN,NaN,NaN
9,MALLRGVFVVAAKRTPFGAYGGLLKDFTATDLSEFAAKAALSAGKV...,A,ACAA2,WT,NaN,NaN,NaN
11,MASSFLPAGAITGDSGGELSSGDDSGEVEFPHSPEIEETSCLAELF...,A,ACBD6,WT,NaN,NaN,NaN
...,...,...,...,...,...,...,...
16068,MIKSQESLTLEDVAVEFTWEEWQLLGPAQKDLY*,Y,ZNF613,stop_gained,ENST00000293471,C3N-01520__C__T,chr19:51940293
16069,MDLPALLPAPTARGGQHGGGPGPLRRAPAPLGASPARRRLLLVRGP...,Y,ZXDC,stop_gained,ENST00000389709,C3L-00771__G__A,chr3:126459707
16070,MATRSSRRESRLPFLFTLVALLPPGALCEVWTQRLHGGSAPLPQDR...,Y,SORL1,frameshift_variant,ENST00000260197,C3L-00908__-__AAAAAAAA,chr11:121605195-121605196
16071,MAGEVSAATGRFSLERLGLPGLALAAALLLLALCLLVRRTRRPGEP...,Y,CYP7B1,stop_gained,ENST00000310193,C3N-00545__C__A,chr8:64615949


In [89]:
ccle_aa_df

,prot_seq,last_aa,gene,Phenotype,Feature,#Uploaded_variation,Location
0,MEGSGGGAGERAPLLGARRAAAAAAAAGAFAGRRAACGAVLLTELL...,A,SLC15A4,WT,NaN,NaN,NaN
19,MACARPLISVYSEKGESSGKNVTLPAVFKAPIRPDIVNFVHTNLRK...,A,RPL4,WT,NaN,NaN,NaN
37,MAATASAGVPATVSEKQEFYQLLKNLINPSCMVRRQAEEIYENIPG...,A,RANBP6,WT,NaN,NaN,NaN
60,MKNQDKKNGAAKQSNPKSSPGQPEAGPEGAQERPSQAAPAVEAEGP...,A,TXLNA,WT,NaN,NaN,NaN
81,MGDAADPREMRKTFIVPAIKPFDHYDFSRAKIACNLAWLVAKAFGT...,A,CAMSAP2,WT,NaN,NaN,NaN
...,...,...,...,...,...,...,...
14650,MTTPRNSVNGTFPAEPMKGPIAMQSGPKPLFRRMSSLVGPTQSFFM...,Y,MS4A1,frameshift_variant,ENST00000534668,ACH-000817__-__A,chr11:60468274-60468275
14651,MTTPRNSVNGTFPAEPMKGPIAMQSGPKPLFRRMSSLVGPTQSFFM...,Y,MS4A1,frameshift_variant,ENST00000534668,ACH-000388__-__A,chr11:60468274-60468275
14652,MTTPRNSVNGTFPAEPMKGPIAMQSGPKPLFRRMSSLVGPTQSFFM...,Y,MS4A1,frameshift_variant,ENST00000534668,ACH-000151__-__A,chr11:60468274-60468275
14653,MTTPRNSVNGTFPAEPMKGPIAMQSGPKPLFRRMSSLVGPTQSFFM...,Y,MS4A1,frameshift_variant,ENST00000534668,ACH-000611__-__A,chr11:60468274-60468275


In [87]:
#Save table with last aa annotation
cptac_aa_df.to_csv(r'last_aa_annot_cptac.tsv', header = True, index = None, sep = '\t')
ccle_aa_df.to_csv(r'last_aa_annot_ccle.tsv', header = True, index = None, sep = '\t')